In [101]:
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm
from random import randrange

In [102]:
def img_to_imgs(file):
    size = 4000 # about 1.5 minutes

    img = mpimg.imread(file)
    
    width = img.shape[1]
    split_num = int(np.floor(width/size))
    residual = width - split_num*size
    end_split_index = width - residual
    
    imgs = []
    if split_num > 0:
        imgs = np.array_split(img[:, :end_split_index, :], split_num, axis=1)

    if residual > 60: # about 1.5 seconds
        residual_img = img[:, end_split_index:, :]
        residual_img = np.pad(img, ((0, 0), (0, randrange(7, 13, 1)), (0, 0)), 'constant')
        repeats = int(np.ceil(size/residual_img.shape[1]))
        
        imgs.append(np.tile(residual_img, (1, repeats, 1))[:, :size, :])
    
    return imgs

In [103]:
def split_save_imgs(file, bird_storage_dir):
    file_label = file.split('/')[-1].split('.')[0]
    
    imgs = img_to_imgs(file)

    for i, img in enumerate(imgs):
        mpimg.imsave(bird_storage_dir + "/" + file_label + '-' + str(i + 1)  + '.jpeg', img)

In [104]:
def save_bird_imgs(files, bird_storage_dir): 
    if not os.path.exists(bird_storage_dir):
        os.mkdir(bird_storage_dir)

    for file in files:
        split_save_imgs(file, bird_storage_dir)

In [105]:
working_dir = "./data/train_img/"
birds = []
flist = {} 

for root, dirs, files in os.walk(working_dir):
    if root == working_dir:
        birds = dirs

for i, bird in enumerate(birds):
    blist = [] 
    
    for root, dirs, files in os.walk(working_dir + bird):
        for file in files:
            if file.endswith(".jpeg") and 'ipynb_checkpoints' not in root:
                blist.append(os.path.join(root, file))

    blist.sort()
    flist[bird] = blist

In [106]:
num_cores = multiprocessing.cpu_count()

In [107]:
import warnings
warnings.filterwarnings('ignore')

storage_dir = "./data/train_img_split/"
birds_handled = 0

for bird, files in flist.items():
    bird_dir = storage_dir + bird
    
    if not os.path.exists(bird_dir):
        os.mkdir(bird_dir)

    birds_handled += 1
    print(birds_handled)

    Parallel(n_jobs=num_cores - 2)(
        delayed(split_save_imgs)(file, bird_dir) for file in files
    )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
